## process pickle result for statistics info

In [2]:
import os
import pickle
# load pickle
class my_pickle:
    def __init__(self, file_path) -> None:
        with open(file_path, 'rb') as f:
            result = pickle.load(f)
        if isinstance(result, list):
            version = "old"
        elif isinstance(result, dict):
            version = "new"
        else:
            raise ValueError("not dict or list type")
        if version == "old":
            [self.correct_num, self.error_num, self.answers_list, self.contents_list] = result
        else:
            self.correct_num = result["correct_num"]
            self.error_num = result["error_num"]
            self.answers_list = result["answers_list"]
            self.contents_list = result["contents_list"]
        
    def avg_acc(self, case_num, reason_num):
        correct_num = 0
        error_num = 0
        for case_id in range(case_num):
            case_answers = self.answers_list[case_id]
            for answer_id in range(reason_num):
                answer = case_answers[answer_id]
                if isinstance(answer, list):
                    if answer[1] == 1:
                        correct_num += 1  
                    else: 
                        error_num += 1
        print("acc:{}, correct num: {}, err num: {}".format(correct_num / (error_num + correct_num), correct_num, error_num))
        
        return correct_num / (error_num + correct_num), correct_num, error_num
    
    def vote(self, case_num, reason_num):
        from collections import Counter
        valid_count = 0
        vote_right_count = 0
        for case_id in range(case_num):
            answers = self.answers_list[case_id][:reason_num]
            answers = [answer for answer in answers if isinstance(answer, list)]  # clean answers
            if len(answers) == 0:
                continue
            else:
                valid_count += 1
            second_elements_are_1 = [sublist[1] == 1 for sublist in answers]
            any_second_element_is_1 = any(second_elements_are_1)
            if not any_second_element_is_1:
                continue
            true_answer = next((sublist[0] for sublist in answers if sublist[1] == 1), None)
            counter = Counter(sublist[0] for sublist in answers)
            guess_value, _ = counter.most_common(1)[0]
            if guess_value == true_answer:
                vote_right_count += 1

        print("vote_correct_num:{}, vaild_num:{}, vote_correct_num:{}".format(vote_right_count, valid_count, vote_right_count / valid_count))        
        

In [60]:
dataset = "SCAN"
sub_task = "simple"
method = "baseline"
case_num = 500
if_ICL = True
shot_num = 3
if_noise = True
noise_type = "inaccurate"
noise_ratio = 0.8



In [61]:
if not if_ICL:
    file_path = f"./result/{dataset}/{sub_task}/gpt-3.5-turbo-0613/method_{method}/log_origin_case{case_num}"
else:    
    if not if_noise:
        file_path = f"./result/{dataset}/{sub_task}/gpt-3.5-turbo-0613/method_{method}/log_ICL_{shot_num}clean_origin_case{case_num}"
        
    else:
        file_path = f"./result/{dataset}/{sub_task}/gpt-3.5-turbo-0613/method_{method}/log_ICL_0clean_noise_3{noise_type}_fixed_ratio{noise_ratio}_case{case_num}"
if method == "baseline":
    file_path += "_temp1_n10"
file_path += ".pkl"
print(file_path)
pkl = my_pickle(file_path)
print(pkl.correct_num, pkl.error_num)

./result/SCAN/simple/gpt-3.5-turbo-0613/method_baseline/log_ICL_0clean_noise_3inaccurate_fixed_ratio0.8_case500_temp1_n10.pkl
1184 3804


In [62]:
# save_case_num = 300
# save_reason_num = 5
pkl.avg_acc(300, 10)
# file_path =  f"./result/base_math/base-9/gpt-3.5-turbo-0613/method_baseline/log_ICL_0clean_noise_3{noise_type}_fixed_ratio{noise_ratio}_case{save_case_num}_temp1_n{save_reason_num}.log"

acc:0.22719679251587035, correct num: 680, err num: 2313


(0.22719679251587035, 680, 2313)

In [63]:
# for vote
pkl.vote(300, 5)

vote_correct_num:78, vaild_num:300, vote_correct_num:0.26
